In [1]:
# Importing necessary libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

#定义标签
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes",
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins",
    "logged_in", "num_compromised", "root_shell", "su_attempted", "num_root",
    "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
    "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"
]

#读文件
file_path = r'C:\Users\22275\Desktop\NSL_KDD-master\KDDTrain+.csv'
data = pd.read_csv(file_path, names=column_names, index_col=False)
data.head(5)

C:\Users\22275\AppData\Local\Temp\ipykernel_28184\3073725935.py:22: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  data = pd.read_csv(file_path, names=column_names, index_col=False)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [2]:
#简单的数据预处理
#空缺值填充
data['duration'] = pd.to_numeric(data['duration'], errors='coerce').fillna(0)
data['dst_host_srv_rerror_rate'] = pd.to_numeric(data['dst_host_srv_rerror_rate'], errors='coerce').fillna(0)
#转换为数值型
categorical_columns = ["protocol_type", "service", "flag"]
for column in categorical_columns:
    data[column] = LabelEncoder().fit_transform(data[column])
#分开x与y
X = data.drop('label', axis=1)
y = data['label']
y = LabelEncoder().fit_transform(y)
scaler = StandardScaler()
#进行数据归一化
X_scaled = scaler.fit_transform(X)
print(y)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train[:1], y_train[:5]
#这里label其实可以不用换成数字，为了方便对应换的y = LabelEncoder().fit_transform(y)这一句 这里的11是normal

[11 11  9 ... 11  9 11]


(array([[-1.10249223e-01, -1.24706157e-01, -4.42083085e-01,
          7.51111287e-01, -7.72578609e-03, -1.20363334e-03,
         -1.40888118e-02, -8.94864220e-02, -7.73598503e-03,
         -9.50756715e-02, -2.70228184e-02,  1.23569403e+00,
         -1.16636426e-02, -3.66518691e-02, -2.44365073e-02,
         -1.23851504e-02, -2.61800242e-02, -1.86098963e-02,
         -4.12211976e-02,  0.00000000e+00, -2.81749392e-03,
         -9.75309440e-02, -5.94783455e-01, -1.34065042e-01,
         -6.37209268e-01, -6.31929033e-01, -3.74362240e-01,
         -3.74431603e-01,  7.71283106e-01, -3.49683031e-01,
          2.79715587e-01, -1.31190837e+00,  1.25875427e+00,
          1.06640135e+00, -4.39078168e-01, -4.15471070e-01,
          2.43929738e-01, -6.39531905e-01, -6.24870800e-01,
         -3.87634624e-01, -3.76387026e-01]]),
 array([11,  9,  9, 11, 11]))

In [3]:
#相关性分析，初步想法提取前十五个相关性系数高的进行训练，后续可以再次进行调整
X_with_label = pd.DataFrame(X_scaled, columns=X.columns)
X_with_label['label'] = y
correlation_matrix = X_with_label.corr()
label_correlation = correlation_matrix['label'].drop('label', axis=0).sort_values(ascending=False)
print("特征与 'label' 的相关性系数（由高到低）:")
print(label_correlation)

特征与 'label' 的相关性系数（由高到低）:
srv_count                      0.310819
wrong_fragment                 0.304125
dst_host_diff_srv_rate         0.295042
same_srv_rate                  0.258357
diff_srv_rate                  0.228557
dst_host_same_src_port_rate    0.202229
flag                           0.170749
dst_host_srv_count             0.138848
protocol_type                  0.135203
duration                       0.134590
is_guest_login                 0.109112
dst_host_rerror_rate           0.108057
dst_host_srv_rerror_rate       0.106217
logged_in                      0.104056
rerror_rate                    0.099397
hot                            0.098611
srv_rerror_rate                0.098105
dst_host_same_srv_rate         0.073548
count                          0.060380
dst_host_count                 0.053869
src_bytes                      0.011617
dst_bytes                      0.007600
num_access_files               0.005220
su_attempted                   0.004005
num_file_creat

In [4]:
#自编码器
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split


In [5]:
X_train

array([[-0.11024922, -0.12470616, -0.44208309, ..., -0.6248708 ,
        -0.38763462, -0.37638703],
       [-0.11024922, -0.12470616,  1.08730518, ...,  1.6189552 ,
        -0.38763462, -0.37638703],
       [-0.11024922, -0.12470616, -0.44208309, ..., -0.6248708 ,
         2.87440955,  2.75391372],
       ...,
       [-0.11024922, -0.12470616, -0.44208309, ..., -0.6248708 ,
         2.87440955,  2.75391372],
       [-0.11024922, -0.12470616, -0.44208309, ..., -0.6248708 ,
        -0.38763462, -0.37638703],
       [-0.11024922,  2.21931185, -1.17618945, ..., -0.6248708 ,
        -0.38763462, -0.37638703]])

In [6]:
#网络，这个网络效果不是很好，需要修改
class ResidualBlock(nn.Module):
    def __init__(self, n_features):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(n_features, n_features)
        self.elu = nn.ELU()
        self.fc2 = nn.Linear(n_features, n_features)

    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.elu(out)
        out = self.fc2(out)
        out += residual  # 残差连接
        out = self.elu(out)
        return out

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(len(top_features), 128),
            nn.ELU(),
            ResidualBlock(128),
            nn.Linear(128, 64),
            nn.ELU(),
            ResidualBlock(64),
            nn.Linear(64, 3))

        self.decoder = nn.Sequential(
            nn.Linear(3, 64),
            nn.ELU(),
            ResidualBlock(64),
            nn.Linear(64, 128),
            nn.ELU(),
            ResidualBlock(128),
            nn.Linear(128, len(top_features)),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
# 创建一个包含列名的 DataFrame
X_with_label = pd.DataFrame(X_scaled, columns=X.columns)
X_with_label['label'] = y

# 计算相关性
correlation_matrix = X_with_label.corr()
label_correlation = correlation_matrix['label'].drop('label', axis=0).sort_values(ascending=False)
top_features = label_correlation.nlargest(15).index.tolist()
feature_indices = [X_with_label.columns.get_loc(col) for col in top_features]
X_train_top_features = X_train[:, feature_indices]
normal_indices = y_train == 11
X_train_top_features_normal = X_train_top_features[normal_indices]
X_train_tensor = torch.tensor(X_train_top_features_normal, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
#这里我只使用了normal数据进行训练，想法是这样便于识别，不知道理解有没有出问题

In [8]:
# # 筛选出正常数据的标签
# y_train_normal = y_train[normal_indices]

# # 定义要使用的数据比例
# train_size = 0.6
# test_size = 1 - train_size

# # 切分数据为训练集和验证集
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_top_features_normal, y_train_normal, train_size=train_size, test_size=test_size, random_state=42)

# # 转换为 PyTorch 张量
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# X_val_tensor = torch.tensor(X_val, dtype=torch.float32)

# # 创建数据集
# train_dataset = TensorDataset(X_train_tensor, X_train_tensor)
# val_dataset = TensorDataset(X_val_tensor, X_val_tensor)

# # 创建数据加载器
# train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)
# val_loader = DataLoader(dataset=val_dataset, batch_size=256, shuffle=False)

# # 现在 train_loader 和 val_loader 可以用于训练和验证



In [9]:
X_train_tensor.shape

torch.Size([53921, 15])

In [11]:
# 检查是否有可用的GPU，并相应地设置device变量
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
# 实例化模型并移动到device上
autoencoder = Autoencoder().to(device)
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(autoencoder.parameters(), lr=0.003)

# 训练模型
num_epochs = 200
autoencoder.train()
for epoch in range(num_epochs):
    total_loss = 0
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader)}')

Using device: cuda
Epoch [1/200], Loss: 0.8244673070184427
Epoch [2/200], Loss: 0.7947531969626368
Epoch [3/200], Loss: 0.7646772451875334
Epoch [4/200], Loss: 0.7333282709686677
Epoch [5/200], Loss: 0.6995434133927404
Epoch [6/200], Loss: 0.666191271413559
Epoch [7/200], Loss: 0.635297519618301
Epoch [8/200], Loss: 0.6105981764352717
Epoch [9/200], Loss: 0.5919250040822684
Epoch [10/200], Loss: 0.5781477689743042
Epoch [11/200], Loss: 0.5669797578411645
Epoch [12/200], Loss: 0.5571020124082882
Epoch [13/200], Loss: 0.5481192133437965
Epoch [14/200], Loss: 0.5389414531359741
Epoch [15/200], Loss: 0.5306208130872645
Epoch [16/200], Loss: 0.5236728906066497
Epoch [17/200], Loss: 0.5181476332565055
Epoch [18/200], Loss: 0.5137433022684396
Epoch [19/200], Loss: 0.5098650514797012
Epoch [20/200], Loss: 0.5066704706275632
Epoch [21/200], Loss: 0.5035975792961663
Epoch [22/200], Loss: 0.500495932819719
Epoch [23/200], Loss: 0.49765779692414813
Epoch [24/200], Loss: 0.49492199245787344
Epoch [

Epoch [197/200], Loss: 0.353556355735137
Epoch [198/200], Loss: 0.35309687200315754
Epoch [199/200], Loss: 0.35295403780530415
Epoch [200/200], Loss: 0.35281497889785407


In [ ]:
#定义CFBPNN学习
#这段没有完成只是找的一个简单的框架
import torch
import torch.nn as nn

class CFBPNN(nn.Module):
    def __init__(self, encoded_feature_size, hidden_size, output_size, cpn_size):
        super(CFBPNN, self).__init__()
        # 反向传播网络部分
        self.backpropagation = nn.Sequential(
            nn.Linear(encoded_feature_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
        # 对流神经网络部分
        self.counterpropagation = nn.Linear(encoded_feature_size, cpn_size)

    def forward(self, x):
        # 编码器部分来自自编码器
        encoded = autoencoder.encoder(x)
        # 反向传播路径
        bp_output = self.backpropagation(encoded)
        # 对流神经网络路径
        cpn_output = self.counterpropagation(encoded)
        return bp_output, cpn_output


In [ ]:
autoencoder.eval()  # 将自编码器设置为评估模式

# 使用自编码器的编码器部分来提取特征
encoded_features = [autoencoder.encoder(inputs).detach() for inputs, _ in train_loader]
encoded_features = torch.cat(encoded_features).numpy()

# 准备用于CFBPNN的数据集
cfbpnn_dataset = TensorDataset(torch.tensor(encoded_features, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))

# 创建数据加载器
cfbpnn_loader = DataLoader(dataset=cfbpnn_dataset, batch_size=256, shuffle=True)

# 定义、训练和评估CFBPNN模型的代码应该在这里
